In [9]:
#!pip install sklearn
!pip install scikit-learn

Defaulting to user installation because normal site-packages is not writeable
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 79.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 302.2/302.2 kB 19.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 36.4/36.4 MB 49.5 MB/s eta 0:00:0000:0100:01


In [1]:
from huggingface_hub import notebook_login
notebook_login()

In [2]:
import torch

In [3]:
############# code changes ###############
import intel_extension_for_pytorch as ipex
############# code changes ###############

In [11]:
from datasets import DatasetDict, Dataset

In [12]:
import pandas as pd
df_train = pd.read_pickle("data/train_df_new_wa.pkl")
df_test = pd.read_pickle("data/validation_df_new_wa.pkl")

In [13]:
df_train['is_question'] = df_train['Answer'].apply(lambda x: 1 if x.lower() == 'yes' or x.lower() == 'no' else 0)
df_train = df_train[['question', 'is_question']].rename(columns={'question': 'text', 'is_question': 'label'})

df_test['is_question'] = df_test['Answer'].apply(lambda x: 1 if x.lower() == 'yes' or x.lower() == 'no' else 0)
df_test = df_test[['question', 'is_question']].rename(columns={'question': 'text', 'is_question': 'label'})

In [14]:
dataset_dict = DatasetDict({
    'train': Dataset.from_pandas(df_train),
    'test': Dataset.from_pandas(df_test)
})

In [15]:
dataset_dict

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 59062
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 6563
    })
})

In [16]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("distilbert-base-uncased")

In [17]:
def preprocess_function(examples):
    return tokenizer(examples["text"], truncation=True)

In [18]:
tokenized_dataset_dict = dataset_dict.map(preprocess_function, batched=True)

Map:   0%|          | 0/59062 [00:00<?, ? examples/s]

Map:   0%|          | 0/6563 [00:00<?, ? examples/s]

In [19]:
from transformers import DataCollatorWithPadding
data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

**Evaluate**

In [20]:
import evaluate
accuracy = evaluate.load("accuracy")

In [21]:
import numpy as np

def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    return accuracy.compute(predictions=predictions, references=labels)

In [23]:
id2label = {0: "NO", 1: "YES"}
label2id = {"YES": 0, "NO": 1}

In [24]:
from transformers import AutoModelForSequenceClassification, TrainingArguments, Trainer

model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", num_labels=2, id2label=id2label, label2id=label2id
)

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.weight', 'pre_classifier.weight', 'classifier.bias', 'pre_classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [25]:
training_args = TrainingArguments(
    output_dir="yn_question",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=2,
    weight_decay=0.01,
    evaluation_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    use_ipex=True,
    fp16=True,
    push_to_hub=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset_dict["train"],
    eval_dataset=tokenized_dataset_dict["test"],
    tokenizer=tokenizer,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
)

trainer.train()

/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/torch/cuda/amp/grad_scaler.py:118: UserWarning: torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.
  warnings.warn("torch.cuda.amp.GradScaler is enabled, but CUDA is not available.  Disabling.")
/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:484: UserWarning: Split Master Weight feature is not supported on XPU for now, disabled.
  warnings.warn("Split Master Weight feature is not supported on XPU for now, disabled.")
/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:500: UserWarning: Weight Prepack and Sample Input are both disabled on XPU. The Onednn Layout is automatically applied.
  warnings.warn(
You're using a DistilBertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text follow

Epoch,Training Loss,Validation Loss,Accuracy
1,0.096200,0.100274,0.964193
2,0.087800,0.102143,0.962974


/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:494: UserWarning: To reduce device memory usage on XPU, optimization are done inplace, setting the inplace argument to True.
  warnings.warn(
/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:527: UserWarning: Linear BatchNorm folding failed during the optimize process.
  warnings.warn("Linear BatchNorm folding failed during the optimize process.")
/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:484: UserWarning: Split Master Weight feature is not supported on XPU for now, disabled.
  warnings.warn("Split Master Weight feature is not supported on XPU for now, disabled.")
/home/common/miniconda3/envs/pytorch_xpu/lib/python3.10/site-packages/intel_extension_for_pytorch/frontend.py:494: UserWarning: To reduce device memory usage on XPU, optimization are done in

TrainOutput(global_step=7384, training_loss=0.0970875237899896, metrics={'train_runtime': 289.1277, 'train_samples_per_second': 408.553, 'train_steps_per_second': 25.539, 'total_flos': 543468718468056.0, 'train_loss': 0.0970875237899896, 'epoch': 2.0})

In [26]:
trainer.push_to_hub()

'https://huggingface.co/roaltopo/yn_question/tree/main/'